# TESScut and ExoMAST: Supplimental Exercises

1. [Explore a different weird TCE](#1.-Explore-a-different-weird-TCE)
2. [Find a known exoplanet in TESS data](#2.-Find-a-known-exoplanet-in-TESS-data)
3. [Look at a flaring star in the TESS field](#3.-Look-at-a-flaring-star-in-the-TESS-field)
4. [Examine stellar variability](#4.-Examine-stellar-variability)

In [ ]:
# For querying for data
import requests
from astroquery.mast import Tesscut

# For manipulating data
import numpy as np

from astropy.table import Table
from astropy.coordinates import SkyCoord

import re

# For matplotlib plotting
import matplotlib
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.animation as animation

# For animation display
from matplotlib import rc
from IPython.display import HTML
rc('animation', html='jshtml')

# For bokeh plotting
from bokeh import plotting
plotting.output_notebook()

## Useful functions from the tutorial

The functions used in the tutorial notebook are included here for easy access.

In [ ]:
def json_to_table(fields, data):
    """"
    Takes a json object and turns it into an astropy table.

    Parameters
    ----------
    fields : list of dicts
        Of the form [{colname:,datatype:,description:}, ...]
    data : list of dicts
       Of the form [{col1:, col2:, ...},{col1:, col2:, ...}, ...]

    Returns
    -------
    response : `astropy.table.Table`
    """

    rx = re.compile(r"varchar\((\d+)\)")
    
    data_table = Table()

    for col, atype in [(x['colname'], x['datatype']) for x in fields]:
        col = col.strip()
        if "varchar" in atype:
            match = rx.search(atype)
            atype = "U" + match.group(1)
        if atype == "real":
            atype = "float"
        data_table[col] = np.array([x.get(col, None) for x in data], dtype=atype)

    return data_table


In [ ]:
def find_index(btjd):
    """
    Given a time as a Barycentric TESS Julian Date (BTJD) timestamp, return the closest index in a table
    that is assumed to have a TIME column that is also in BTJD"""
    
    return (np.abs(cutout_table['TIME'] - btjd)).argmin()

In [ ]:
def make_animation(data_array, start_frame=0, end_frame=None, vmin=None, vmax=None, delay=50):
    """
    Function that takes an array where each frame is a 2D image array and make an animated plot
    that runs through the frames.
    
    Note: This can take a long time to run if you have a lot of frames.    
    Parameters
    ----------
    data_array : array
        Array of 2D images.
    start_frame : int
        The index of the initial frame to show. Default is the first frame.
    end_frame : int
        The index of the final frame to show. Default is the last frame.
    vmin : float
        Data range min for the colormap. Defaults to data minimum value.
    vmax : float
        Data range max for the colormap. Defaults to data maximum value.
    delay: 
        Delay before the next frame is shown in milliseconds.

    Returns
    -------
    response : `animation.FuncAnimation`
    """
    
    if not vmin:
        vmin = np.min(data_array)
    if not vmax:
        vmax = np.max(data_array)
        
    if not end_frame:
        end_frame = len(data_array) - 1 # set to the end of the array
        
    num_frames = end_frame - start_frame + 1 # include the end frame
        
    def animate(i, fig, ax, binarytab, start=0):
        ax.set_title("Epoch #" + str(i+start))
        im = ax.imshow(binarytab[i+start], cmap=plt.cm.YlGnBu_r, vmin=vmin, vmax=vmax)
        return im,
    
    # Create initial plot.
    fig, ax = plt.subplots(figsize=(10,10))
    ax.imshow(data_array[start_frame], cmap=plt.cm.YlGnBu_r, vmin=vmin, vmax=vmax)

    ani = animation.FuncAnimation(fig, animate, fargs=(fig, ax, data_array, start_frame), frames=num_frames, 
                                  interval=delay, repeat_delay=1000)
    
    plt.close()
    
    return ani

## 1. Explore a different weird TCE

These TIC IDs also have TCEs that "look weird"
- 00206462606
- 00206544316

Work through the steps from the tutorial, and see if you can determine what is going on with them.

## 2. Find a known exoplanet in TESS data

The planet WASP-18 b was observed by TESS. Figure out its TIC ID and TCE number, and then pull and plot the associated TESS light curve.

To get you started, the query form for resolving an exoplanet in the exomast API is `exoplanets/identifiers/?name=<planet name>`. Documentation __[here](https://exo.mast.stsci.edu/docs/#resolving-exoplanets)__

## 3. Look at a flaring star in the TESS field

TIC ID 231017428 has shows several flare events. Explore these events in the DV data timeseries, and see if you can make a movie where the largest flare is visible.


## 4. Examine stellar variability

TIC ID 25118964, shows visible variability (and some flares). See if you can make a video that shows this variability.